Importing the dataset

In [1]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Deep Learning/train.csv.zip')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Dropping the Nan values

In [2]:
data = data.dropna()

Defining the dependent and independent variables

In [3]:
X = data.drop('label', axis=1)
y = data['label']

Checking the shape of X, y

In [4]:
print(X.shape)
print(y.shape)

(18285, 4)
(18285,)


Importing the tensorflow libraries

In [5]:
import tensorflow as tf
print(tf.__version__)

2.8.2


Importing the tensorflow libraries

In [6]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dense

Defining the vocabulary size

In [7]:
vocabsize = 5000

One Hot Representation

In [11]:
messages = X.copy()

In [12]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

Resetting the indexes

In [13]:
messages.reset_index(inplace=True)

Import nltk libraries

In [14]:
import nltk 
import re
from nltk.corpus import stopwords

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Data Preprocessing

In [16]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

Checking the corpus

In [17]:
corpus[:20]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

One Hot Representation

In [19]:
onehotrep = [one_hot(words, vocabsize) for words in corpus]
onehotrep[:5]

[[547, 3683, 3059, 1967, 3395, 995, 2765, 3844, 4670, 4165],
 [2891, 1428, 2344, 2696, 3432, 1425, 1100],
 [1851, 4180, 2370, 2197],
 [2192, 4694, 1165, 1813, 864, 771],
 [925, 3432, 960, 106, 544, 1981, 3432, 2477, 278, 274]]

Embedding Representation

In [41]:
sentencelength = 20
embeddeddocs = pad_sequences(onehotrep, padding='pre', maxlen=sentencelength)
print(embeddeddocs)

[[   0    0    0 ... 3844 4670 4165]
 [   0    0    0 ... 3432 1425 1100]
 [   0    0    0 ... 4180 2370 2197]
 ...
 [   0    0    0 ... 4447 3632 2985]
 [   0    0    0 ... 3133 1769 4070]
 [   0    0    0 ... 2941 3329 4588]]


In [42]:
embeddeddocs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  547,
       3683, 3059, 1967, 3395,  995, 2765, 3844, 4670, 4165], dtype=int32)

Creating models

In [43]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocabsize, embedding_vector_features, input_length=sentencelength))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
len(embeddeddocs), y.shape

(18285, (18285,))

In [45]:
import numpy as np
Xfinal = np.array(embeddeddocs)
yfinal = np.array(y)

Train test split

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xfinal, yfinal, test_size=0.33, random_state=42)

Training the model

In [47]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 41ms/step - loss: 0.3354 - accuracy: 0.8478 - val_loss: 0.1957 - val_accuracy: 0.9178
Epoch 2/10
192/192 [==============================] - 10s 50ms/step - loss: 0.1444 - accuracy: 0.9415 - val_loss: 0.1985 - val_accuracy: 0.9225
Epoch 3/10
192/192 [==============================] - 7s 37ms/step - loss: 0.1016 - accuracy: 0.9611 - val_loss: 0.2293 - val_accuracy: 0.9215
Epoch 4/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0775 - accuracy: 0.9714 - val_loss: 0.2612 - val_accuracy: 0.9150
Epoch 5/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0581 - accuracy: 0.9812 - val_loss: 0.3007 - val_accuracy: 0.9125
Epoch 6/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0412 - accuracy: 0.9860 - val_loss: 0.3715 - val_accuracy: 0.9122
Epoch 7/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0297 - accuracy: 0.9897 - val_loss: 0.3865 - val_accuracy: 0.91

Adding Dropout

In [48]:
from tensorflow.keras.layers import Dropout

embedding_vector_features=40
model = Sequential()
model.add(Embedding(vocabsize, embedding_vector_features, input_length=sentencelength))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuarcy'])

Performance metrics and Accuracy

In [49]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

Confusion Matrix

In [50]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3128,  291],
       [1610, 1006]])

Accuracy Score

In [51]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6850041425020712